In [3]:
import sys
#!{sys.executable} -m pip install librosa
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install keras
#!{sys.executable} -m pip install tensorflow
#!{sys.executable} -m pip install scikit-learn
import numpy as np, keras, librosa, tensorflow, scipy

In [4]:
import os

In [5]:
def wav2mfcc(file_path, max_pad_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)
    if (max_pad_len>mfcc.shape[1]):
        pad_width = max_pad_len-mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0),(0,pad_width)),mode='constant')
    else:
        mfcc = mfcc[:,:max_pad_len]
    return mfcc

In [6]:
import numpy as np
DATA_PATH = "/home/aaa/aatresh/Audio Processing/data/"
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, keras.utils.to_categorical(label_indices)

In [7]:
def save_data_to_array(path=DATA_PATH, max_pad_len=11):
    labels,_,_= get_labels(path)
    
    for label in labels:
        mfcc_vectors = []
        
        wavfiles = [path+label+'/'+wavfile for wavfile in os.listdir(path+'/'+label)]
        for wavfile in wavfiles:
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len)
            mfcc_vectors.append(mfcc)
            
        np.save(label+'.npy',mfcc_vectors)

In [8]:
from sklearn.model_selection import train_test_split
def get_train_test(split_ratio=0.6, random_state=42):
    labels, indices,_ = get_labels(DATA_PATH)
    
    X = np.load(labels[0]+'.npy')
    y = np.zeros(X.shape[0])
    for i, label in enumerate(labels[1:]):
        x = np.load(label+'.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i+1)))
        
    assert X.shape[0] == len(y)
    
    return train_test_split(X, y, test_size=(1-split_ratio),random_state=random_state, shuffle=True)

In [9]:
save_data_to_array()

In [10]:
X_train, X_test, y_train, y_test = get_train_test()
X_train = X_train.reshape(X_train.shape[0], 20, 11, 1)
X_test = X_test.reshape(X_test.shape[0], 20, 11, 1)
y_train_hot = keras.utils.to_categorical(y_train)
y_test_hot = keras.utils.to_categorical(y_test)

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
model = Sequential()
model.add(Conv2D(32, kernel_size=(2,2), activation='relu',input_shape=(20,11,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5,activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
model.fit(X_train, y_train_hot,batch_size=100, epochs=300, verbose=1, validation_data=(X_test, y_test_hot))


Instructions for updating:
Use tf.cast instead.
Train on 5746 samples, validate on 3831 samples
Epoch 1/300
5746/5746 [==============================] - 1s 200us/step - loss: 11.4139 - acc: 0.2574 - val_loss: 10.7280 - val_acc: 0.2814
Epoch 2/300
5746/5746 [==============================] - 1s 152us/step - loss: 5.5394 - acc: 0.3474 - val_loss: 1.1006 - val_acc: 0.5244
Epoch 3/300
5746/5746 [==============================] - 1s 151us/step - loss: 1.2879 - acc: 0.4485 - val_loss: 0.9744 - val_acc: 0.5437
Epoch 4/300
5746/5746 [==============================] - 1s 215us/step - loss: 1.1067 - acc: 0.5164 - val_loss: 0.8655 - val_acc: 0.6098
Epoch 5/300
5746/5746 [==============================] - 1s 178us/step - loss: 1.0306 - acc: 0.5538 - val_loss: 0.8299 - val_acc: 0.6599
Epoch 6/300
5746/5746 [==============================] - 1s 171us/step - loss: 0.9503 - acc: 0.6020 - val_loss: 0.7782 - val_acc: 0.6883
Epoch 7/300
5746/5746 [==============================] - 1s 146us/step - loss: 0

5746/5746 [==============================] - 2s 265us/step - loss: 0.1559 - acc: 0.9427 - val_loss: 0.2848 - val_acc: 0.9060
Epoch 118/300
5746/5746 [==============================] - 1s 206us/step - loss: 0.1662 - acc: 0.9391 - val_loss: 0.2826 - val_acc: 0.9045
Epoch 119/300
5746/5746 [==============================] - 1s 165us/step - loss: 0.1720 - acc: 0.9340 - val_loss: 0.2886 - val_acc: 0.9102
Epoch 120/300
5746/5746 [==============================] - 1s 205us/step - loss: 0.1604 - acc: 0.9405 - val_loss: 0.2865 - val_acc: 0.9081
Epoch 121/300
5746/5746 [==============================] - 1s 247us/step - loss: 0.1557 - acc: 0.9391 - val_loss: 0.3429 - val_acc: 0.8893
Epoch 122/300
5746/5746 [==============================] - 1s 180us/step - loss: 0.1591 - acc: 0.9403 - val_loss: 0.2886 - val_acc: 0.9047
Epoch 123/300
5746/5746 [==============================] - 1s 162us/step - loss: 0.1486 - acc: 0.9445 - val_loss: 0.2939 - val_acc: 0.9045
Epoch 124/300
5746/5746 [================

5746/5746 [==============================] - 1s 159us/step - loss: 0.1227 - acc: 0.9572 - val_loss: 0.3125 - val_acc: 0.9089
Epoch 177/300
5746/5746 [==============================] - 1s 164us/step - loss: 0.1285 - acc: 0.9542 - val_loss: 0.3378 - val_acc: 0.9003
Epoch 178/300
5746/5746 [==============================] - 1s 169us/step - loss: 0.1267 - acc: 0.9579 - val_loss: 0.3320 - val_acc: 0.9037
Epoch 179/300
5746/5746 [==============================] - 1s 162us/step - loss: 0.1244 - acc: 0.9565 - val_loss: 0.3201 - val_acc: 0.9039
Epoch 180/300
5746/5746 [==============================] - 1s 158us/step - loss: 0.1239 - acc: 0.9534 - val_loss: 0.3148 - val_acc: 0.9102
Epoch 181/300
5746/5746 [==============================] - 1s 171us/step - loss: 0.1215 - acc: 0.9561 - val_loss: 0.3295 - val_acc: 0.8992
Epoch 182/300
5746/5746 [==============================] - 1s 169us/step - loss: 0.1176 - acc: 0.9581 - val_loss: 0.3083 - val_acc: 0.9068
Epoch 183/300
5746/5746 [================

5746/5746 [==============================] - 1s 182us/step - loss: 0.0970 - acc: 0.9655 - val_loss: 0.3466 - val_acc: 0.9068
Epoch 236/300
5746/5746 [==============================] - 1s 191us/step - loss: 0.0907 - acc: 0.9650 - val_loss: 0.3772 - val_acc: 0.9021
Epoch 237/300
5746/5746 [==============================] - 1s 189us/step - loss: 0.1043 - acc: 0.9650 - val_loss: 0.3679 - val_acc: 0.9052
Epoch 238/300
5746/5746 [==============================] - 1s 192us/step - loss: 0.1001 - acc: 0.9629 - val_loss: 0.3509 - val_acc: 0.9066
Epoch 239/300
5746/5746 [==============================] - 1s 225us/step - loss: 0.1021 - acc: 0.9621 - val_loss: 0.3362 - val_acc: 0.9097
Epoch 240/300
5746/5746 [==============================] - 1s 180us/step - loss: 0.0931 - acc: 0.9666 - val_loss: 0.3421 - val_acc: 0.9042
Epoch 241/300
5746/5746 [==============================] - 1s 212us/step - loss: 0.0961 - acc: 0.9664 - val_loss: 0.3586 - val_acc: 0.9026
Epoch 242/300
5746/5746 [================

5746/5746 [==============================] - 1s 176us/step - loss: 0.0848 - acc: 0.9722 - val_loss: 0.3455 - val_acc: 0.9102
Epoch 295/300
5746/5746 [==============================] - 1s 171us/step - loss: 0.0863 - acc: 0.9692 - val_loss: 0.3453 - val_acc: 0.9128
Epoch 296/300
5746/5746 [==============================] - 1s 170us/step - loss: 0.0885 - acc: 0.9687 - val_loss: 0.3557 - val_acc: 0.9131
Epoch 297/300
5746/5746 [==============================] - 1s 225us/step - loss: 0.0821 - acc: 0.9704 - val_loss: 0.3658 - val_acc: 0.9079
Epoch 298/300
5746/5746 [==============================] - 1s 174us/step - loss: 0.0890 - acc: 0.9690 - val_loss: 0.3727 - val_acc: 0.9050
Epoch 299/300
5746/5746 [==============================] - 1s 166us/step - loss: 0.0818 - acc: 0.9709 - val_loss: 0.3576 - val_acc: 0.9063
Epoch 300/300
5746/5746 [==============================] - 1s 182us/step - loss: 0.0762 - acc: 0.9701 - val_loss: 0.3688 - val_acc: 0.9081


In [13]:
sample = wav2mfcc('/home/aaa/aatresh/Audio Processing/data/go/0a9f9af7_nohash_0.wav')
sample_reshaped = sample.reshape(1,20,11,1)
print(get_labels()[0][np.argmax(model.predict(sample_reshaped))])

go


In [16]:
import socket
import numpy as np
from pygame import mixer
import time
import pyaudio


try:
        mySock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        print ("Socket created")

except socket.error:
        print ("Socket connection failed")

port = 8888
try:
        mySock.connect(("192.168.43.195", port))
        print ("Connected to RPi!")
except socket.error:
        print ("Not able to connect")

        

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "out.wav"
while True:
        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)

        print("* recording")

        frames = []

        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)

        print("* done recording")

        stream.stop_stream()
        stream.close()
        p.terminate()

        wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()
        
        recording = AudioSegment.from_wav("./out.wav")
        recording+=20
        recording.export("./out.wav", format="wav")

        print("Before the sleep statement")
        mixer.init(frequency=44100)
        mixer.music.load("./out.wav")
        mixer.music.play()
        time.sleep(5)
        print("After the sleep statement")
        
        sample = wav2mfcc("./out.wav")
        sample_reshaped = sample.reshape(1,20,11,1)
        message = get_labels("/home/aaa/aatresh/Audio Processing/data/")[0][np.argmax(model.predict(sample_reshaped))]   

        print(message)
        mySock.send(message.encode())
        data = mySock.recv(1024).decode()
        print ("Received from server: " + data)
        


mySock.close()


ModuleNotFoundError: No module named 'pyaudio'